<a href="https://colab.research.google.com/github/vkrisvasan/GenAIAgent/blob/main/AgentEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#build and deploy an agent that uses the Gemini Pro model, VertexAI Reasoning Engine, Python functions as tools, and LangChain for orchestration
#The Agent retrieves the exchange rate between two currencies on a specified date

#Gitaccount: vkrisvasan Repo: GenAIAgent Code: AgentEx1.ipynb
#Simple Agent assist implemented by
  #1. creating a project in google vertexai https://console.cloud.google.com/vertex-ai [note the projectID]
  #2. create storage bucket in https://console.cloud.google.com/storage/browser - use  "us-central1" location [Note location and bucket ID]

In [ ]:
!pip install --upgrade --quiet \
    google-cloud-aiplatform==1.51.0 \
    langchain==0.1.20 \
    langchain-google-vertexai==1.0.3 \
    cloudpickle==3.0.0 \
    pydantic==2.7.1 \
    requests

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import os
import getpass
credential_names = ["GCP_PROJECT_ID", "LOCATION", "STAGING_BUCKET"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GCP_PROJECT_ID··········
Provide your...LOCATION··········
Provide your...STAGING_BUCKET··········


In [ ]:
PROJECT_ID = os.environ["GCP_PROJECT_ID"]
LOCATION = os.environ["LOCATION"]
STAGING_BUCKET = os.environ["STAGING_BUCKET"]
STAGING_BUCKET_FULL = "gs://{}".format(STAGING_BUCKET)


In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET_FULL)

In [ ]:
import requests
from vertexai.preview import reasoning_engines

In [ ]:
model = "gemini-1.5-pro-preview-0409"

In [ ]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [ ]:
get_exchange_rate(currency_from="USD", currency_to="INR")

{'amount': 1.0, 'base': 'USD', 'date': '2024-06-21', 'rates': {'INR': 83.57}}

In [ ]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [ ]:
agent.query(input="What's the exchange rate from US dollars to Indian currency today?")

{'input': "What's the exchange rate from US dollars to Indian currency today?",
 'output': 'The exchange rate from US dollars to Indian rupees is 1 USD to 83.57 INR. \n',
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'],
    'kwargs': {'tool': 'get_exchange_rate',
     'tool_input': {'currency_from': 'USD', 'currency_to': 'INR'},
     'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'INR'}`\n\n\n",
     'type': 'AgentActionMessageLog',
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'additional_kwargs': {'function_call': {'name': 'get_exchange_rate',
          'arguments': '{"currency_from": "USD", "currency_to": "INR"}'}},
        'response_metadata': {'is_blocked': False,
         'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH',
           'p

In [ ]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

In [ ]:
#Create vertexai Managed Runtime (Reasoning Engine): Effortlessly deploy and scale your application using Vertex AI’s secure and reliable infrastructure.
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform==1.51.0",
        "langchain==0.1.20",
        "langchain-google-vertexai==1.0.3",
        "cloudpickle==3.0.0",
        "pydantic==2.7.1",
        "requests",
    ],
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket agent_kv_ex1
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://agent_kv_ex1/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://agent_kv_ex1/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://agent_kv_ex1/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/17529158352/locations/us-central1/reasoningEngines/7374257361726210048/operations/2815373499437154304
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/17529158352/locations/us-central1/reasoningEngines/7374257361726210048
INFO:vertexai.reasoning_engines._

In [ ]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Indian currency today?"
)

{'input': "What's the exchange rate from US dollars to Indian currency today?",
 'output': 'The exchange rate from US dollars to Indian rupees is 1 USD to 83.57 INR. \n'}

In [ ]:
remote_agent.query(
    input="explain ai"
)

{'output': 'AI, or Artificial Intelligence, is a broad term that refers to the ability of computer systems to perform tasks that typically require human intelligence, such as:\n\n* **Learning:** Acquiring knowledge and skills from data or experiences.\n* **Reasoning:** Using logic and rules to solve problems and make decisions.\n* **Problem-solving:** Finding solutions to complex or challenging situations.\n* **Perception:** Interpreting sensory information, such as images, sounds, and text.\n* **Natural Language Processing:** Understanding and generating human language.\n\nAI encompasses a wide range of techniques and approaches, including:\n\n* **Machine Learning:** Algorithms that allow computers to learn from data without explicit programming.\n* **Deep Learning:** A subset of machine learning that uses artificial neural networks with multiple layers.\n* **Computer Vision:** Enabling computers to "see" and interpret images.\n* **Robotics:** Designing and building robots that can pe